In [ ]:
import json
import re
import cni_connectors.adls_gen1_connector as connector
import pyspark.sql.functions as f
import crawler.functions as cf
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, LongType, DoubleType

In [ ]:
var_adls_uri = connector.adls_gen1_connect(spark, dbutils, scope="adls_gen2", dynamic_overwrite="dynamic")

fornecedor_dev
dev-execution

In [ ]:
#Parametros de entrado do ADF
var_file = {'namespace':'API_InGuru',
'file_folder':'inguro_investimento',
'extension':'csv',
'column_delimiter':',',
'encoding':'UTF-8',
'null_value':''}

#Caminho de Dev
var_dls = {
    "folders":{"landing":"/tmp/dev/uld",
    "error":"/tmp/dev/err",
    "staging":"/tmp/dev/stg",
    "log":"/tmp/dev/log",
    "raw":"/tmp/dev/raw",
    "trusted":"/tmp/dev/trs",
    "business":"/tmp/dev/biz",
    "landing":"/tmp/dev/uld"          }}
#Parametros do ADF (Nativo)
var_adf = {
    "adf_factory_name": "cnibigdatafactory",
    "adf_pipeline_name": "org_raw_investimento",
    "adf_pipeline_run_id": "60ee3485-4a56-4ad1-99ae-666666666",
    "adf_trigger_id": "62bee9e9-acbb-49cc-80f2-666666666",
    "adf_trigger_name": "62bee9e9-acbb-49cc-80f2-66666666",
    "adf_trigger_time": "2020-06-08T01:42:41.5507749Z",
    "adf_trigger_type": "PipelineActivity"
}

In [ ]:
#Acesso ao json do CMD
lnd = var_dls['folders']['landing']
raw = var_dls['folders']['raw']

In [ ]:
var_source = "{lnd}/{namespace}/{file_folder}/".format(lnd=lnd, namespace=var_file['namespace'], file_folder=var_file['file_folder'])
var_source

Out[5]: '/tmp/dev/uld/API_InGuru/inguro_investimento/'

In [ ]:
#Origem e destino
source_lnd = "{adl_path}{lnd}/{namespace}/{file_folder}".format(adl_path=var_adls_uri, lnd=lnd, namespace=var_file["namespace"],file_folder=var_file["file_folder"])
source_lnd

Out[6]: '[REDACTED]/tmp/dev/uld/API_InGuru/inguro_investimento'

In [ ]:
url = url_nodeID, per_page=1, start_date='', end_date='')['data'][0]

In [1]:
def API_INGURU(url, per_page, start_date, end_date):
    headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
    'accept':'application/json',
    'authorization': '6f64d6a047fad369d35c3806f8f8e0560475075a432585973f91caae35b4ad74'
    } 

    params = {
        'per_page': 1,
        'start_date': '',
        'end_date': '',
        'sort': '1'
    }
    response = requests.get(url, headers=headers, params=params)
    return response.json()

In [ ]:
def divide_data(data_inicial, data_final):
    formato_data = '%d/%m/%Y %H:%M'

    data_inicial = datetime.strptime(data_inicial, formato_data)
    data_final = datetime.strptime(data_final, formato_data)
    intervalo = data_final - data_inicial
    data_final = data_inicial + intervalo // 2
    data_final = data_final.replace(hour=23, minute=59, second=59) # define o final como o último segundo do minuto
    data_final = data_final.strftime(formato_data)
    data_inicial = data_inicial.strftime(formato_data)
    return (data_inicial, data_final)

In [ ]:
def load_big_df(path):
  schema = StructType([
    StructField("id", LongType(), nullable=True),
    StructField("domain", StringType(), nullable=True),
    StructField("title", StringType(), nullable=True),
    StructField("subtitle", StringType(), nullable=True),
    StructField("author", StringType(), nullable=True),
    StructField("content", StringType(), nullable=True),
    StructField("url", StringType(), nullable=True),
    StructField("source", StringType(), nullable=True),
    StructField("source_country", StringType(), nullable=True),
    StructField("source_state", StringType(), nullable=True),
    StructField("crawled_date", StringType(), nullable=True),
    StructField("published_date", StringType(), nullable=True),
    StructField("dh_insertion_raw", StringType(), nullable=True)
  ])

  df = spark.read.schema(schema) \
             .option("mergeSchema", "true") \
             .option("timestampFormat", "INT96") \
             .parquet(path) \
             .withColumn("dh_insertion_raw", f.col("dh_insertion_raw").cast("timestamp"))\
             .withColumn("crawled_date", f.col("crawled_date").cast("timestamp"))
  
  return df

In [ ]:
def insere_news(data_inicial, data_final, id, path):
  url = 'https://app.inguru.me/api/v1/taxonomies/nodes/news/'+str(id)

  array_dfs = []

  getted_news = 0
    
  total_news = API_INGURU(url, 1, data_inicial, data_final)['pagination']['total']
    
  if total_news:
      
#     print('Total de notícias: ', total_news)

    parcial_data_inicial = data_inicial
    parcial_data_final = data_final

    while getted_news < total_news:

      parcial_news = total_news

      while parcial_news > 10000:
#         print('Parcial de notícias: ', parcial_news)
        parcial_data_inicial, parcial_data_final = divide_data(parcial_data_inicial, parcial_data_final)
        parcial_news = API_INGURU(url, 1, parcial_data_inicial, parcial_data_final)['pagination']['total']

#       print("fazendo requisicao")
#       print('Data inicial: ', parcial_data_inicial)
#       print('Data final: ', parcial_data_final)
      news = API_INGURU(url, parcial_news, parcial_data_inicial, parcial_data_final)['data']

      #CRIA UM DATAFRAME COM AS NOTICIAS
      noticias = pd.DataFrame(news)

      #corrige as datas
      parcial_data_inicial = parcial_data_final
      parcial_data_final = data_final

      getted_news += parcial_news

      dh_insertion_raw = var_adf["adf_trigger_time"].split(".")[0]

      schema = StructType([
        StructField("id", LongType(), nullable=True),
        StructField("domain", StringType(), nullable=True),
        StructField("title", StringType(), nullable=True),
        StructField("subtitle", StringType(), nullable=True),
        StructField("author", StringType(), nullable=True),
        StructField("content", StringType(), nullable=True),
        StructField("url", StringType(), nullable=True),
        StructField("source", StringType(), nullable=True),
        StructField("source_country", StringType(), nullable=True),
        StructField("source_state", StringType(), nullable=True),
        StructField("crawled_date", StringType(), nullable=True),
        StructField("published_date", StringType(), nullable=True)
      ])

      saveDF = spark.createDataFrame(noticias, schema)
      saveDF = saveDF.withColumn("dh_insertion_raw", f.lit(dh_insertion_raw).cast("string"))
      saveDF.write.parquet(path, "append")
      print("Total:", total_news)
      print("Inseridas:", getted_news)
    
  else:
    print("Sem dados")


In [ ]:
graph_ids = []
graph_names = []
graph_dfs = []

nodeID = '820' #Declaração do ID

url_nodeID = 'https://app.inguru.me/api/v1/taxonomies/nodes/'+str(nodeID)
IDs = API_INGURU(url = url_nodeID, per_page=1, start_date='', end_date='')['data'][0]
graph_ids.append(IDs['id'])
graph_names.append(IDs['name'].replace(" ", "_"))

#Passa por cada "filho" no nó principal, caso não tenha nenhum, nada acontece.
for node_children in IDs['children']:
  graph_ids.append(node_children['id'])
  graph_names.append(node_children['name'].replace(" ", "_"))

data_inicial = '01/01/2015 00:00'
data_final = datetime.now().strftime("%d/%m/%Y %H:%M")

for i, id in enumerate(graph_ids):
  path = source_lnd + "_" + str(graph_names[i])
  insere_news(data_inicial, data_final, id, path)


Total: 4718
Inseridas: 4718

In [ ]:
print(path)

[REDACTED]/tmp/dev/uld/API_InGuru/inguro_investimento_Investimento

In [ ]:
# CARREGAMENTO DAS TABELAS PARA VER SE FORAM SALVAS CORRETAMENTE
test_dfs = []
for i, df in enumerate(graph_dfs):
  path = source_lnd + "_" + str(graph_names[i])
  test_dfs.append(load_big_df(path))
  print(path)

In [ ]:
print(test_dfs)

[]